**Sommario**

- [`flask_login_3` - *Template inheritance* e *Toast*](#flask_login_3---template-inheritance-e-toast)
  - [Struttura del progetto aggiornata](#struttura-del-progetto-aggiornata)
  - [Ereditarietà dei template](#ereditariet%C3%A0-dei-template)
    - [24. Utilizzo della *template inheritance* con `{% block %}` e `{% extends %}`](#24-utilizzo-della-template-inheritance-con--block--e--extends)
      - [Creazione del template base](#creazione-del-template-base)
      - [Estensione del template base nei template figli](#estensione-del-template-base-nei-template-figli)
  - [Flash message con il Toast](#flash-message-con-il-toast)
    - [25. Utilizzo dei *Toast* di Bootstrap per i messaggi `flash`](#25-utilizzo-dei-toast-di-bootstrap-per-i-messaggi-flash)
    - [26. Script JavaScript per i toast, dalla cartella `static/`](#26-script-javascript-per-i-toast-dalla-cartella-static)
  - [Code refactoring](#code-refactoring)
    - [27. Ri-organizzazione del codice secondo paradigma MVC](#27-ri-organizzazione-del-codice-secondo-paradigma-mvc)

# `flask_login_3` - *Template inheritance* e *Toast*

## Struttura del progetto aggiornata

In questa quarta fase modifichiamo ancora una volta la struttura del progetto in questo modo:

```text
flask_login_3/
│
├── app.py
├── db.py                      << NUOVO
├── static/
│   ├── script.js
│   ├── style.css
│   └── imgs/
│       ├── akira.jpg
│       ├── blade-runner.jpg
│       ├── gits.jpg
│       ├── hackers.jpg
│       ├── nirvana.jpg
│       └── shortcut-icon.png  
└── templates/
    ├── base.html              << NUOVO
    ├── films.html
    ├── home.html
    ├── login.html
    └── includes/
        ├── flash.html
        ├── footer.html
        ├── head.html
        └── navbar.html
```


## Ereditarietà dei template

### 24. Utilizzo della *template inheritance* con `{% block %}` e `{% extends %}`

L'ereditarietà dei template in Jinja2 permette di definire un template base che altri template possono estendere. Questo aiuta a mantenere il codice [DRY (Don't Repeat Yourself)](https://it.wikipedia.org/wiki/Don%27t_repeat_yourself) e facilita la manutenzione.

#### Creazione del template base

Creiamo un file `templates/base.html` in cui andremo ad inserire:

- `{% block title %}- Sito Film{% endblock %}` nel tag `<title>`;

- `{% block content %}{% endblock %}` nel tag `<main>`

```html
<!doctype html>
<html>
<head>
    <!-- BLOCCO DEL TITOLO -->
    <title>{% block title %}- Sito Film{% endblock %}</title>

    {% include 'includes/head.html' %}
</head>
<body>

    <header>
        {% include 'includes/navbar.html' %}
    </header>
    
    <main>
        <div class="container mt-4">
        {% include 'includes/flash.html' %}

        <!-- BLOCCO DEL CONTENUTO -->
        {% block content %}{% endblock %}
    
        </div>
    </main>
    {% include 'includes/footer.html' %}

</body>
</html>
```

#### Estensione del template base nei template figli

I template figli utilizzano la sintassi `{% extends 'template_base.html'%}` per ereditare il template base e `{% block nome_blocco %}...{% endblock %}` per definire il contenuto specifico che andrà inserito nel template base.

Riscriviamo ciascun file:

- `templates/home.html`
- `templates/login.html`
- `templates/films.html`

E vi inseriamo:

- `{% extends 'base.html' %}` all'inizio del file per indicare che si sta estendendo il template `base.html`;

- `{% block title %} ... {% endblock %}` con all'interno il TITOLO vero e proprio della pagina;

- `{% block content %} ... {% endblock %}` con all'interno il CONTENUTO vero e proprio della pagina.

Ad esempio:

```html
{% extends 'base.html' %}

{% block title %}Home - Sito Films{% endblock %}

{% block content %}
  <h1 class="display-4">Benventuti su Sito Film!</h1>
  <p class="lead">Questa è la home page del sito.</p>
  <!-- ... altro contenuto -->
{% endblock %}
```

## Flash message con il Toast

### 25. Utilizzo dei *Toast* di Bootstrap per i messaggi `flash`

I *toast* di Bootstrap offrono un modo elegante e user-friendly per mostrare messaggi temporanei agli utenti.

Modifichiamo il file `templates/includes/flash.html` in questo modo:

```html
{% with messages = get_flashed_messages(with_categories=true) %}
    {% if messages %}
    <div class="position-relative" aria-live="polite" aria-atomic="true">
        <!-- Container per il posizionamento del Toast Bootstrap -->
        <div class="toast-container position-fixed top-1 end-0 p-4">
            {% for category, message in messages %}
            <div class="toast align-items-center text-bg-{{ category }} border-0 fade hide" role="alert" aria-live="assertive" aria-atomic="true">
                <div class="d-flex">
                    <div class="toast-body">
                        {{ message }}
                    </div>
                    <button type="button" class="btn-close btn-close-white me-2 m-auto" data-bs-dismiss="toast" aria-label="Close"></button>
                </div>
            </div>
            {% endfor %}
        </div>
    </div>
    {% endif %}
{% endwith %}
```

Ora ci serve un po' di JavaScript.

### 26. Script JavaScript per i toast, dalla cartella `static/`

Per far funzionare correttamente i toast, dobbiamo aggiungere del codice JavaScript.

Creiamo uno script `script.js` nella cartella `static/`:

```javascript
// Per precauzione, attivo il toast solo dopo che la pagina è stata caricata completamente
document.addEventListener('DOMContentLoaded', () => {
    const toastElList = document.querySelectorAll('.toast')
    const toastList = [...toastElList].map(toastEl => new bootstrap.Toast(toastEl, {
        autohide: true,  // Si nasconde automaticamente dopo 'delay' tempo
        delay: 5000      // in millisecondi ( == 5 secondi )
    }).show());
});
```

> **NOTA**: Questo codice codice l'ho preso [dal sito ufficiale di Bootstrap](https://getbootstrap.com/docs/5.3/components/toasts/#usage).

Ora nel template parziale `includes/footer.html` bisogna aggiungere l'importazione dello script:

```html
<script src="{{ url_for('static', filename='script.js') }}"></script>
```

## Code refactoring

### 27. Ri-organizzazione del codice secondo paradigma MVC

La modularità del codice è importante per mantenere il progetto organizzato e facile da mantenere. 

Iniziamo ad apportare un'importante separazione, utile nel paradigma [MVC (Model-View-Controller)](https://it.wikipedia.org/wiki/Model-view-controller): quella tra il CONTROLLER (la logica che controlla le route, i comportamenti del sito) e il MODEL (i dati). La VIEW (i file HTML), è già separata.

Creiamo dunque un file separato per la gestione del nostro "finto database", ad esempio `db.py`, e ci mettiamo le strutture dati che abbiamo creato per contenere gli utenti e i film. Nel prossimo step leggeremo questi dati direttamente da un database.

File `db.py`:

```python
USERS = {
    'mrossi': 'osoejfj3',
    'ggangi': 'odoeooeee'
}

FILMS = [
    {'title': 'Akira', 'image': 'akira.jpg'},
    {'title': 'Ghost in the Shell', 'image': 'gits.jpg'},
    {'title': 'Blade Runner', 'image': 'blade-runner.jpg'},
    {'title': 'Hackers', 'image': 'hackers.jpg'},
    {'title': 'Nirvana', 'image': 'nirvana.jpg'},
]
```

Ora dobbiamo aggiornare `app.py` per utilizzare il nuovo modulo `db`:

```python
from flask import Flask, ...
from db import USERS, FILMS  # Importiamo i "modelli" con i dati
...

# Ora possiamo usare USERS
if rx_username in USERS and rx_password == USERS[rx_username]:
    ...

...
# E possiamo usare anche FILMS
return render_template('films.html', films=FILMS)
...
```